https://github.com/Thinkful-Ed/data-201-assignment-solutions/blob/master/model_prep_missing_values/solution.ipynb

```
# Print all values that cannot be converted to float

for column_name in ["Video Uploads", "Subscribers"]:
    print("These are the problematic values for the variable: {}".format(column_name))
    for value in youtube_df[column_name]:
        try:
            float(value)
        except:
            print(value)
```

```
# Replace `--` values with empty strings

youtube_df["Video Uploads"] = youtube_df["Video Uploads"].apply(str.strip).replace("--", np.nan)
youtube_df["Video Uploads"] = pd.to_numeric(youtube_df["Video Uploads"], downcast="float")

youtube_df["Subscribers"] = youtube_df["Subscribers"].apply(str.strip).replace("--", np.nan)
youtube_df["Subscribers"] = pd.to_numeric(youtube_df["Subscribers"], downcast="float")
```

```
# Percent missing values

youtube_df.isnull().sum()*100/youtube_df.isnull().count()
```

```
youtube_df.Grade.value_counts()
```

```
youtube_df.Grade = youtube_df.Grade.apply(str.strip).replace("", np.nan)

youtube_df.Grade.unique()
```

```
youtube_df["Video Uploads"].fillna(youtube_df["Video Uploads"].mean(), inplace=True)
youtube_df["Subscribers"].fillna(youtube_df["Subscribers"].mean(), inplace=True)

# Be careful with `inplace=True`. It works in this case, but sometimes, it's safer to use this:
# youtube_df["Video Uploads"] = youtube_df["Video Uploads"].fillna(youtube_df["Video Uploads"].mean())
# youtube_df["Subscribers"] = youtube_df["Subscribers"].fillna(youtube_df["Subscribers"].mean())
# (See the assessment's example solution for more detail on this issue.)
```

```
# Fill in missing values in `Grade` with the value of the next-highest-ranked channel's category

grade_list = youtube_df.Grade

for i in range(0, len(youtube_df.Grade)):
    if pd.isnull(youtube_df.Grade[i]):
        youtube_df.Grade[i] = youtube_df.Grade[i-1]
        
youtube_df["Grade"].unique()
```

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import seaborn as sns
import matplotlib as plt

warnings.filterwarnings('ignore')

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

educ_df = pd.read_sql_query('select * from useducation',con=engine)

# No need for an open connection, 
# because you're only doing a single query
engine.dispose()

1. Determine all the variable types and find the fraction of the missing values for each variable.



In [ ]:
educ_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

In [ ]:
round(educ_df.isnull().sum()/educ_df.isnull().count(),3)

PRIMARY_KEY                     0.000
STATE                           0.000
YEAR                            0.000
ENROLL                          0.176
TOTAL_REVENUE                   0.142
FEDERAL_REVENUE                 0.142
STATE_REVENUE                   0.142
LOCAL_REVENUE                   0.142
TOTAL_EXPENDITURE               0.142
INSTRUCTION_EXPENDITURE         0.142
SUPPORT_SERVICES_EXPENDITURE    0.142
OTHER_EXPENDITURE               0.176
CAPITAL_OUTLAY_EXPENDITURE      0.142
GRADES_PK_G                     0.116
GRADES_KG_G                     0.088
GRADES_4_G                      0.088
GRADES_8_G                      0.088
GRADES_12_G                     0.088
GRADES_1_8_G                    0.088
GRADES_9_12_G                   0.088
GRADES_ALL_G                    0.116
AVG_MATH_4_SCORE                0.641
AVG_MATH_8_SCORE                0.643
AVG_READING_4_SCORE             0.643
AVG_READING_8_SCORE             0.666
dtype: float64


#2.   Notice that the data has a time dimension (year). For this assessment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variable. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

In [ ]:
educ2_df = educ_df.copy()

ENROLL: The U.S. Census Bureau's count for students in the state. Should be comparable to GRADES_ALL (which is the
NCES's estimate for students in the state).

In [ ]:
round(educ2_df['ENROLL'].describe(),2)

count       1229.00
mean      915930.82
std      1065280.42
min        43866.00
25%       258314.00
50%       648313.00
75%      1014528.00
max      6307022.00
Name: ENROLL, dtype: float64

In [ ]:
educ2_df['ENROLL'].isnull().sum()

263

In [ ]:
#Replace with GRADES_ALL when possible
educ2_df['ENROLL'].fillna(value=educ2_df['GRADES_ALL_G'], inplace=True)

In [ ]:
#112 Missing values remain
educ2_df['ENROLL'].isnull().sum()

112

In [ ]:
round(educ2_df['ENROLL'].describe(),2)

count       1380.00
mean      853808.29
std      1039367.65
min         7254.00
25%       189734.00
50%       557814.00
75%       976131.75
max      6307022.00
Name: ENROLL, dtype: float64

In [ ]:
#educ_df[(educ_df['ENROLL'].isnull()) & (educ_df['YEAR']==2017)]
nas = educ2_df[educ2_df['ENROLL'].isnull()]
nas.sort_values(by=['YEAR']).head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
51,1992_WYOMING,WYOMING,1992,NaN,629748.0,31714.0,314216.0,283818.0,624941.0,339953.0,185930.0,NaN,60815.0,NaN,7184.0,8049.0,8078.0,6613.0,64614.0,28515.0,NaN,225.377592,275.080076,221.185556,NaN
1425,1992_VIRGIN_ISLANDS,VIRGIN_ISLANDS,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1680.0,1878.0,1549.0,1117.0,14422.0,5590.0,NaN,NaN,NaN,NaN,NaN
1404,1993_VIRGIN_ISLANDS,VIRGIN_ISLANDS,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1624.0,1755.0,1568.0,1097.0,14312.0,5508.0,NaN,NaN,NaN,NaN,NaN
1368,1994_VIRGIN_ISLANDS,VIRGIN_ISLANDS,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1487.0,1745.0,1585.0,1140.0,14329.0,5838.0,NaN,NaN,NaN,NaN,NaN
1340,1995_VIRGIN_ISLANDS,VIRGIN_ISLANDS,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1638.0,1655.0,1607.0,1102.0,13854.0,5756.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
#replace all with mean
educ2_df['ENROLL'].fillna(educ2_df['ENROLL'].mean())

0       6.761740e+05
1       1.123350e+05
2       6.148810e+05
3       4.052590e+05
4       4.717112e+06
            ...     
1487    8.538083e+05
1488    8.538083e+05
1489    8.538083e+05
1490    8.538083e+05
1491    8.538083e+05
Name: ENROLL, Length: 1492, dtype: float64

In [ ]:
columns = educ2_df.columns
new_columns = columns[2:]
for column in new_columns:
  educ2_df.loc[:, column] = educ2_df.loc[:, column].fillna(educ2_df.loc[:, column].mean())

In [ ]:
educ2_df['ENROLL'].head()

0     676174.0
1     112335.0
2     614881.0
3     405259.0
4    4717112.0
Name: ENROLL, dtype: float64

In [ ]:
for col in fill_list:
    education2_df.loc[:, col] = education2_df.loc[:, col].fillna(education2_df.loc[:, col].mean())

NameError: ignored

3.   Now, take into account the time factor. Replicate your second answer, but this time, fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill in a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.



In [ ]:
educ3_df = educ_df.copy()

In [ ]:
educ3_df['YEAR'].unique()

In [ ]:
columns = educ3_df.columns
new_columns = columns[2:]
years = educ3_df['YEAR'].unique()
for column in new_columns:
  for year in years:
    educ3_df.loc[educ3_df["YEAR"] == year, column] = educ3_df.loc[educ3_df["YEAR"] == year, column].fillna(
        educ3_df[educ3_df["YEAR"] == year][column].mean())

In [ ]:
education3_df = education_df.copy()

years = education3_df["YEAR"].unique()

for col in fill_list:
    for year in years:
        education3_df.loc[education3_df["YEAR"] == year, col] = education3_df.loc[education3_df["YEAR"] == year, col].fillna(
            education3_df[education3_df["YEAR"] == year][col].mean())

In [ ]:
educ3_df[educ3_df["YEAR"] == 2000]["ENROLL"].mean()

In [ ]:
educ3_df.loc[educ3_df["YEAR"]==2000,"ENROLL"].mean()

In [ ]:
educ3_df.loc[educ3_df["YEAR"]==2000]["ENROLL"].mean()



4.   This time, fill in the missing values using interpolation (extrapolation).





In [ ]:
educ4_df = educ_df.copy()


In [ ]:
educ4_df['ENROLL'].interpolate()

In [ ]:
educ4_df = educ_df.copy()

for column in columns:
    educ4_df.loc[:, column] = educ4_df.loc[:, column].interpolate()

5.   Compare your results for the second, third, and fourth questions. Do you find any meaningful differences?


In [ ]:
educ2_df.isnull().sum()

In [ ]:
educ3_df.isnull().sum()

In [ ]:
educ4_df.isnull().sum()

In [ ]:
educ2_df.describe()

In [ ]:
for col in fill_list:
    print("Statistics for columns: {}".format(col))
    print(pd.concat([education_df[col], education2_df[col],
                     education3_df[col], education4_df[col]], axis=1).describe())


In [ ]:
fig = plt.figure(figsize=(20,100))

for i, col in enumerate(fill_list):
    df = pd.concat([education_df[col], education2_df[col],
                     education3_df[col], education4_df[col]], axis=1)
    
    plt.subplot(len(fill_list), 4, i+1)
    plt.bar(["original", "overall", "year-by-year", "interpolation"], df.median())
    plt.xticks(rotation='vertical')
    plt.title("Mod of {}".format(col))
    
plt.tight_layout()
plt.show()